## Alpha-Beta Pruning

In this assignment, we will implement the Alpha-Beta pruning algorithm to find the Alpha and Beta values for the nodes in the Min/Max game tree below:

In [1]:
graph = {
    'a0': ['b0', 'b1'],
    'b0': ['c0', 'c1'],
    'b1': ['c2', 'c3'],
    'c0': ['d0', 'd1', 'd2'],
    'c1': ['d3', 'd4'],
    'c2': ['d5', 'd6'],
    'c3': ['d7', 'd8'],
    'd0': ['e0'],
    'd1': ['e1', 'e2', 'e3'],
    'd2': ['e4'],
    'd3': ['e5', 'e6'],
    'd4': ['e7', 'e8'],
    'd5': ['e9', 'e10'],
    'd6': ['e11'],
    'd7': ['e12'],
    'd8': ['e13', 'e14']
}

The final nodes identified by the letter 'e' are the terminal nodes and these are their states:

In [2]:
terminal_nodes = {
    'e0': -1, 
    'e1': 2,
    'e2': 3, 
    'e3': -5,
    'e4': 0, 
    'e5': 4,
    'e6': 7, 
    'e7': 2,
    'e8': 6, 
    'e9': 9,
    'e10': 8, 
    'e11': 1,
    'e12': 4, 
    'e13': 0,
    'e14': 5 
}

### Implementation

We will do a recurrent version of the algorithm where our stopping condition is opening a terminal node. All alphas and betas are going to be saved in a list of lists called 'alphas_betas'. Also, to be consistent with AIMA's implementation (using +/-infinity) our max and min initial values will be defined using math's library 'inf'. Finally, we will print all expanded and pruned nodes.

In [3]:
from math import inf

alphas_betas = []

In [4]:
# Recursive implementation of alpha-beta pruning algorithm. The final value of the root 
# will be returned. It also prints all expanded and pruned nodes and saves all alphas and 
# betas for each node in a global list. max_player is true when it is max player's turn.
def alphabeta(node, alpha, beta, max_player):
    if node in terminal_nodes:
        print("Opening Terminal Node", node, "->", terminal_nodes[node])
        return terminal_nodes[node]
    
    num_node_child = len(graph[node])
    if max_player:
        v = -inf
        print("Expanding Max Node", node, "->", "alpha:", alpha,
              "beta:", beta)       
        for num_child, child in enumerate(graph[node]):
            v = max(v, alphabeta(child, alpha, beta, False))
            alpha = max(alpha, v)
            alphas_betas.append([node, [alpha, beta]]) # Saving alpha-beta values
            if (beta <= alpha):
                if (num_child < num_node_child-1):
                    for i in range(num_child+1, num_node_child):
                        print("Pruning Node -> ", graph[node][i])
                break
        return v
    else:
        v = inf
        print("Expanding Min Node", node, "->", "alpha:", alpha,
              "beta:", beta)       
        for num_child, child in enumerate(graph[node]):
            v = min(v, alphabeta(child, alpha, beta, True))
            beta = min(beta, v)
            alphas_betas.append([node, [alpha, beta]]) # Saving alpha-beta values
            if (beta <= alpha):
                if (num_child < num_node_child-1):
                    for i in range(num_child+1, num_node_child):
                        print("Pruning Node -> ", graph[node][i])
                break
        return v   

### Output

Now we will use our implementation. The input is our initial node 'a0', initial alpha -inf, initial beta +inf, and max_player=true (it is max player's turn).

Additionally, we are going to find the Best move, the Move that the root would make. The way we will do it is the following: we will find our a0 (root) final alpha (max value) among the final betas (min values) of a0's children. It is true that just a simple 'if statement' might be enough to get the best move of 2 children, but we try to generalize the problem, so that we can use this code in other cases (n children):

In [5]:
max_node_value = alphabeta('a0', -inf, inf, True)
print("\nExpansions completed")
print("\nValue of Max Node a0 = ", max_node_value)

## Best move
final_alpha_betas = dict(alphas_betas)
possible_moves = graph['a0']
best_alpha = -inf
best_move = ''
for move in possible_moves:
    beta = final_alpha_betas[move][1]
    if (beta > best_alpha):
        best_alpha = beta
        best_move = move
print("Move that Max Node a0 would make is -> ", best_move)

Expanding Max Node a0 -> alpha: -inf beta: inf
Expanding Min Node b0 -> alpha: -inf beta: inf
Expanding Max Node c0 -> alpha: -inf beta: inf
Expanding Min Node d0 -> alpha: -inf beta: inf
Opening Terminal Node e0 -> -1
Expanding Min Node d1 -> alpha: -1 beta: inf
Opening Terminal Node e1 -> 2
Opening Terminal Node e2 -> 3
Opening Terminal Node e3 -> -5
Expanding Min Node d2 -> alpha: -1 beta: inf
Opening Terminal Node e4 -> 0
Expanding Max Node c1 -> alpha: -inf beta: 0
Expanding Min Node d3 -> alpha: -inf beta: 0
Opening Terminal Node e5 -> 4
Opening Terminal Node e6 -> 7
Pruning Node ->  d4
Expanding Min Node b1 -> alpha: 0 beta: inf
Expanding Max Node c2 -> alpha: 0 beta: inf
Expanding Min Node d5 -> alpha: 0 beta: inf
Opening Terminal Node e9 -> 9
Opening Terminal Node e10 -> 8
Expanding Min Node d6 -> alpha: 8 beta: inf
Opening Terminal Node e11 -> 1
Expanding Max Node c3 -> alpha: 0 beta: 8
Expanding Min Node d7 -> alpha: 0 beta: 8
Opening Terminal Node e12 -> 4
Expanding Min Nod

### Additional Notes in alphas and betas

The alphas and betas printed above are the alphas and betas at the moment we expand a certain node. In order to better understand the alpha-beta algorithm, it is very helpful to take a look to how the alphas and betas change for each node as we traverse the tree in a depth-first search fashion. Obviously, these nodes are non-pruned nodes.

In [6]:
alphas_betas

[['d0', [-inf, -1]],
 ['c0', [-1, inf]],
 ['d1', [-1, 2]],
 ['d1', [-1, 2]],
 ['d1', [-1, -5]],
 ['c0', [-1, inf]],
 ['d2', [-1, 0]],
 ['c0', [0, inf]],
 ['b0', [-inf, 0]],
 ['d3', [-inf, 0]],
 ['d3', [-inf, 0]],
 ['c1', [4, 0]],
 ['b0', [-inf, 0]],
 ['a0', [0, inf]],
 ['d5', [0, 9]],
 ['d5', [0, 8]],
 ['c2', [8, inf]],
 ['d6', [8, 1]],
 ['c2', [8, inf]],
 ['b1', [0, 8]],
 ['d7', [0, 4]],
 ['c3', [4, 8]],
 ['d8', [4, 0]],
 ['c3', [4, 8]],
 ['b1', [0, 4]],
 ['a0', [4, inf]]]

Also, the final alphas and betas values for each node in the tree could help us know the best path in each turn.

In [7]:
final_alpha_betas

{'d0': [-inf, -1],
 'c0': [0, inf],
 'd1': [-1, -5],
 'd2': [-1, 0],
 'b0': [-inf, 0],
 'd3': [-inf, 0],
 'c1': [4, 0],
 'a0': [4, inf],
 'd5': [0, 8],
 'c2': [8, inf],
 'd6': [8, 1],
 'b1': [0, 4],
 'd7': [0, 4],
 'c3': [4, 8],
 'd8': [4, 0]}